In [1]:
from pyearth import Earth
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import StratifiedKFold, KFold, StratifiedShuffleSplit
from sklearn.base import clone
from pyearth import Earth

In [2]:
names = 'Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,' +        'Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,' +        'Horizontal_Distance_To_Fire_Points,W_Rawah,W_Neota,W_Comanche,W_Cache,2702,2703,2704,2705,2706,' +        '2717,3501,3502,4201,4703,4704,4744,4758,5101,5151,6101,6102,6731,7101,7102,7103,7201,7202,7700,' +        '7701,7702,7709,7710,7745,7746,7755,7756,7757,7790,8703,8707,8708,8771,8772,8776,Cover_Type'
names = names.split(',')
data = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz',
    compression='gzip', names=names)
X = data.drop('Cover_Type', axis=1).values
y = data['Cover_Type'].values

In [3]:
print(X.shape, y.shape)

((581012, 54), (581012,))


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=10000, random_state=59)
print(X_train.shape, X_test.shape)

((10000, 54), (571012, 54))


In [14]:
%%time
earth  = Earth(use_fast=False, fast_h=5, fast_K=1, max_terms=30)
earth.fit(X_train, y_train)

CPU times: user 1min 45s, sys: 10.8 s, total: 1min 56s
Wall time: 14.3 s


In [15]:
p = {
    "n_jobs": -1,
    "bootstrap": True,
    "min_samples_leaf": 6,
    "n_estimators": 73,
    "criterion": "gini",
    "min_samples_split": 7
}   
clf = RandomForestClassifier(**p)
clf.fit(X_train, y_train)
print(X_train.shape)
(clf.predict(X_test)!= y_test).mean()

(10000, 54)


0.23431206349428735

In [ ]:
skf = StratifiedKFold(y, n_folds=5, shuffle=True, random_state=42)
#skf = KFold(len(y), n_folds=5, random_state=42)
results = []
for train, test in skf:
    clf_ = RandomForestClassifier(**p)
    clf_.fit(X[train], y[train])
    print(len(train), len(test))
    print((clf_.predict(X[test]) != y[test]).mean())